On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
from pylab import *    # charge toutes les bibliothèques scientifiques
from scipy.optimize import curve_fit    # permet de modéliser selon un modèle personnalisé
from lmfit import minimize,Parameters,Parameter,report_fit
from lmfit.models import ExpressionModel
import re

# Si on veut un graphique interactif, décommenter la ligne avec widget, et commenter la ligne avec inline.
#%matplotlib widget
%matplotlib ipympl
rcParams['figure.figsize'] = [16, 8]    # pour une fenetre graphique adaptée
rcParams['font.size'] = 15    # pour un texte plus grand
rcParams['lines.markersize'] = 15    # pour des symboles plus grands
rcParams['lines.markeredgewidth'] = 2    # pour des symboles plus larges

Création de tableaux de valeurs automatiques (rampe et valeurs aléatoires) :

Importation de tableaux enregistrés localement en fichiers csv, avec attribution **automatique** des noms de variables :

Importation de tableaux enregistrés localement en fichiers csv, avec attribution **manuelle** des noms de variables :

Création manuelle de tableaux de valeurs :

In [ ]:
x = array([])
y = array([])

Calcul des variations, utile pour calculer des dérivées :

Calcul des caractéristiques de la courbe de régression. Attention : NE PAS SUPPRIMER, et ne le modifier que si les paramètres par défaut ne donnent pas de bon résultat :

In [ ]:
def Modele(expression, x, y):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)

    #modele.set_param_hint('a', value = 1, vary = True, min = -inf, max = inf)    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    parametres = modele.make_params()
    
    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value:.3g} ; incertitude : {resultat.params[key].stderr:.2g}\n"
    return(modele, resultat.params, valeurs, expression)

def Calcul_modele(abscisse,  ordonnee, equation, debut, fin, debutCourbe, finCourbe):
    ## On commence par arranger les expressions avec "x" comme variable, car lmfit travaille avec x par défaut :
    ord = ordonnee
    eq = equation
    equation = re.sub(r"\b"+abscisse+r"\b","x", equation)
    abscisse = globals()[abscisse]
    ordonnee = globals()[ordonnee]
    if debutCourbe == None :
        debutCourbe = min(abscisse)
    if finCourbe == None :
        finCourbe = max(abscisse)
    
    # On calcule maintenant les grandeurs liées au modèle :
    xMod = linspace(debutCourbe, finCourbe, 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)
    modele, parametres, valeurs, expression = Modele(equation, abscisse[debut:fin], ordonnee[debut:fin])    # Expression du modèle, et variables en abscisse et en ordonnée ; on peut changer les plages de valeurs à utiliser pour le calcul
    expression = f"{ord} = {eq}"
    yMod = modele.eval(parametres, x = xMod)
    
    # Les paramètres sont définis de façon globale, et utilisables dans des calculs :
    for key in parametres:
        globals()[key] = parametres[key].value
    
    return(xMod, yMod, expression, valeurs, abscisse, ordonnee)

Tracé de graphique(s) à partir des tableaux précédents :

In [ ]:
abscisse = "x"    # choix de la grandeur en abscisse, pour le graphique et pour le modèle
ordonnee = "y"    # choix de la grandeur en ordonnée, pour le graphique et pour le modèle
equation = "a * x + b"    # expression de l'équation correspondant au modèle recherché ; la variable peut être x ou le nom donné en abscisse
debut = None    # indice du premier élément à prendre en compte pour le modèle (None : le premier de la liste)
fin = None    # indice du dernier élément à prendre en compte pour le modèle (None : le dernier de la liste)
debutCourbe = None    # valeur de l'abscisse où débute le tracé du modèle (None : la plus petite valeur de l'abscisse)
finCourbe = None    # valeur de l'abscisse où se termine le tracé du modèle (None : la plus grande valeur de l'abscisse)

subplot(1,1,1)    # création d'une fenêtre de graphique : lignes, colonnes, numéro
xMod, yMod, expression, valeurs, abscisse, ordonnee = Calcul_modele(abscisse, ordonnee, equation, debut, fin, debutCourbe, finCourbe)    # Calcul des valeurs du modèle, ne pas modifier

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}" )    # trace la courbe du modèle

#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Mon titre")    # mettre le titre désiré entre les guillemets
xlabel("")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.

Calcul de valeurs particulières :